# JETBOT Gesture Controlling

The document is run to display camera feed and set up a TCP/IP socket that receives commands from a client. The JetBot works in 2 modes, STOPMODE and DRIVEMODE .if you raise both of your hand and show your palm with your fingers open, it goes into  STOPMODE ; if you show "V sign" with both of your hands, it will get into Drive mode. In STOPMODE if you close all of one hand's fingers and the other hand wide open Jetbot will turn in the direction of the closed hand in its place. In DRIVEMODE left hand is used for acceleration and right hand is used for changing direction, if left had thumb is up the robot will go forward and if the thumb is down it will go downward. If the left-hand's thumb points to other directions (left or right) it won't move. While the left hand is used for acceleration the right hand is used for direction, if the right hand's thumb is up it will go straight and if the thumb is left or right it will go into that direction.
Run all cells and the Jetbot will work properly. 

This document is influenced by :

https://www.computervision.zone/

https://github.com/HannesRingblom/JETANK_gesture_control


### Import dependencies and displaying camera feed

Running the code below will instanciate the camera and link the feed to a display widget.

In [1]:
import socket
import threading
import time
from time import sleep
from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

# Creating a connection between the Jetson camera and a display widget
camera = Camera.instance(width=300, height=300)

image = widgets.Image(format='jpeg', width=500, height=500)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Functions & Variables
 Below all functions and variables of the program, regarding the movement of the robot are listed 

In [2]:
robot = Robot()
serverPort = 2740
serverPort2 = 1234

# create TCP socket and bind to specified port
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind(('', serverPort))
s.listen()

print ("TCP SERVER LISTENING")

def move_forward():
    robot.forward(0.3)
    time.sleep(0.3)
    robot.forward(0.2)
    time.sleep(0.3)
def stop():
    robot.stop()
    
def stopp():
    robot.stop()

def move_backward():
    robot.backward(0.3)
    time.sleep(0.3)
    robot.backward(0.2)
    time.sleep(0.3)

def move_left():
    robot.left_motor.value =-0.3
    robot.right_motor.value =0.3
    time.sleep(0.1)


def move_right():
    robot.left_motor.value = 0.3
    robot.right_motor.value =-0.3
    time.sleep(0.1)
    
def up_right():
    robot.left_motor.value  =0.3
    robot.right_motor.value =0.2
    time.sleep(0.1)
    
def up_left():
    robot.left_motor.value  =0.2
    robot.right_motor.value =0.3
    time.sleep(0.1)
    
def down_right():
    robot.left_motor.value  =-0.3
    robot.right_motor.value =-0.15
    time.sleep(0.1)
def down_left():
    robot.left_motor.value  =-0.15
    robot.right_motor.value =-0.3
    time.sleep(0.1)
        


# Calls move functions respective to the input
def moveJetBot(button_pressed):
    if not controlBot.new_pressed:
        stop()
    else:
        if button_pressed == "f":
            move_forward()
        elif button_pressed == "b":
            move_backward()
        elif button_pressed == "l":
            move_left()
        elif button_pressed == "r":
            move_right()
        elif button_pressed == "s":
            stopp()
        elif button_pressed == "t":
            up_right()
        elif button_pressed == "u":
            up_left()
        elif button_pressed == "o":
            down_right()
        elif button_pressed == "p":
            down_left()
    controlBot.new_pressed = 0
    

sleep(3.5)


TCP SERVER LISTENING


## Control Thread
Create controlThread responsible for the movement of the robot.

In [3]:
class controlThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(controlThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()
        self.button_pressed = ""
        self.new_pressed = 0
        self.mode_pressed = 0

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            self.__flag.wait()

            # Calls functions based on variable set in Input thread
            if self.new_pressed:
                moveJetBot(self.button_pressed)
            if not self.new_pressed and not self.mode_pressed:
                self.pause()
            sleep(0.1)


# Instantiate and start the multi-threaded servo control thread.
controlBot = controlThread()
controlBot.start()

## Input Thread
Create inputThread responsible for the socket connection and input received

In [4]:
class inputThread(threading.Thread):
    def __init__(self, *args, **kwargs):
        super(inputThread, self).__init__(*args, **kwargs)
        self.__flag = threading.Event()
        self.__flag.clear()

    def pause(self):
        self.__flag.clear()

    def resume(self):
        self.__flag.set()

    def run(self):
        while 1:
            conn, addr = s.accept()
            with conn:
                print("Connected to: ")
                print(addr)
                while 1:
                    
                    self.__flag.wait()
                    message, clientAddress = conn.recvfrom(2048)
                    mess = message.decode()
                    controlBot.button_pressed = mess
                    controlBot.new_pressed = 1
                    controlBot.resume()
                    



# Instantiate the gamepad input thread and start reading information
inputThreading = inputThread()
inputThreading.start()
inputThreading.resume()


In [5]:
#camera.stop()

Connected to: 
('192.168.178.27', 13784)
